# <center> 🤼‍♀️ Разминаемся (Задание 1)
В качестве первого задания вам предстоит достать значения средних зарплат по городам России. Сайт, на котором они хранятся: https://stepik.org/media/attachments/lesson/866758/mean_salary_by_city.html 

В качестве ответа получите .csv файл, в котором города идут в алфавитном порядке.

Пример ответа:

<center> <img src='./images/answer_example_1.png' width="550" >

# <center> 🌍 Парсим autonet.ru 🚗

В этой задаче вам предстоит спарсить дополнительные данные по моделям машин с сайта: http://www.autonet.ru/auto/ttx и добавить их в существующий датасет. Для того чтобы было проще понимать, в каком месте вы ошибаетесь (если такое происходит), мы разбили эту задачу на несколько степов, где постепенно будем парсить данный сайт (по 1 баллу за каждый верно выполненный шаг).

<center> <img src='./images/image_v21.png' width="550" >

## <center> 🕊 Собираем ссылки 🔗

Для того чтобы начать доставать информацию о моделях, нужно вначале получить ссылки на эти модели. На первой странице сайта находится список всех марок. Вам нужно:

1) Проитерироваться по всем ссылкам, которые находятся на странице.

2) Отобрать среди них те, которые содержат в себе '/auto/ttx/', так как остальные ссылки не ведут на марки моделей и не интересуют нас.

3) Перейти по каждой из этих ссылок (внизу скриншот того, как выглядит сайт для какой-то конкретной марки) и уже для него повторить пункты 1 и 2, но уже для моделей, а не марок.

<center> <img src='./images/image_v22.png' width="550" >

В итоге у вас должно найтись 1820 ссылок, которые выглядят примерно так:



<center> <img src='./images/image_v23.png' width="550" >

В качестве ответа на эту задачу загрузите датасет с ними в .csv формате (при сохранении используйте index=False). Ссылки должны идти в отсортированном порядке. Пример сабмита: 

<center> <img src='./images/image_v24.png' width="250" >

## <center> 👉👈 Матчим ссылки с датасетом

Отлично! Мы получили ссылки на все марки машин, которые есть на сайте, но, на самом деле, нам нужно спарсить только малую их часть, так как многие марки не отображены в изначальном датасете. В этом задании вам предстоит сопоставить ссылки и машины из датасета `quickstart_train.csv`. 

In [9]:
import pandas as pd

path = 'https://stepik.org/media/attachments/lesson/866758/quickstart_train.csv'

df = pd.read_csv(path)
df.head(3)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,109.99,another_bug,4.737759,1.214131e+07,0.1,180.855726,0.023174,174,170
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,34.48,electro_bug,4.480517,1.803909e+07,0.0,187.862734,12.306011,174,174
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,34.93,gear_stick,4.768391,1.588366e+07,0.1,102.382857,2.513319,174,173


<center> <img src='./images/image_v31.png' width="250" >

У каждой модели первые два слова отвечают за модель и марку, соответственно. Придумайте, как можно соотнести ссылки и наши записи в датафрейме и добавьте их в колонку url. Загрузите получившийся файл в систему (строки должны идти в том же порядке, что и в исходном датасете). Пример результата: 

<center> <img src='./images/image_v32.png' width="350" >

**Примечание 1:** К некоторым моделям может не существовать ссылки. Если данная марка не представлена на сайте, оставьте поле url для них пустым (заполните np.nan).

**Примечание 2:** Некоторые марки могут матчиться не так просто, их придется отдельно заif'ать.

## <center> ⚗️ Достаем технические характеристики

Достаем информацию о машинах

Отлично! Ссылки мы достали, теперь пришло время получить необходимые данные из них. Это скриншот того, как выглядит страничка сайта для конкретной модели (в нашем случае Mitsubishi Galant):

<center> <img src='./images/image_v41.png' width="450" >

Отсюда вам необходимо для каждой подмарки (модификации) получить следующую информацию:

1) `carcass` - тип кузова (в приведенном выше примере это "седан" либо "хэтчбек"); 
2) `doors` - кол-во дверей (целое число);
3) `volume` - объем двигателя (целое число);
4) `power` - мощность в лошадиных силах (целое число);
5) `release_year` - год выпуска (первая из чиселок, которые записаны в графе выпуск);
6) `name` - название из графы "Марка и модификация".
Это все можно сделать при помощи BeautifulSoap.

Соединяем с нашим датасетом

Отлично! Мы достали необходимые данные, теперь осталось соединить это с нашим датасетом. У нас в изначальном датасете есть поле `year_to_start`, которое означает год, в котором начали проводить замеры для данной машины. Было бы странно, если модель, которую мы пытаемся заматчить была выпущена после того, как ей начали пользоваться в рамках какого-то замера. Поэтому пусть у нас есть список всех модификаций, которые потенциально подходят под нашу машину (у них год `release_year <= year_to_start`).

Вот пример подходящих модификация для `car_id=T81007701m` (у которой `year_to_start=2015` и ссылка на модель:  http://www.autonet.ru/auto/ttx/audi/a3):

<center> <img src='./images/image_v42.png' width="450" >

Теперь отсортируйте получившийся датасет по следующим полям:

`df = df.sort_values(by=['name', 'release_year', 'carcass', 'doors', 'volume', 'power'])` и возьмите первую запись в качестве ответа для данной машины. Получившийся датасет загрузите в качестве решения в систему (не изменяйте порядок строк в изначальном датасете).

Пример результата:

<center> <img src='./images/image_v43.png' width="450" >

**Примечание:** Убедитесь, что name и carcass не содержат лишних пробелов вначале и в конце.

